In [9]:
import cv2
import torch
import torch.nn as nn
import numpy as np
from facenet_pytorch import MTCNN
from PIL import Image 

In [10]:
hand_dict = {0: "g is",
             1: 'c',
             2: 'thumb',
             3: 'hello!',
             4: 'no pasaran',
             5: 'ok',
             6: 'no pasaran',
             7: 'index',
             8: 'down',
             9: 'hello!'}

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, 2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(64, 64, 3)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(64*2*8, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)   

        x = self.conv3(x)
        x = self.relu(x)
        x = self.pool(x)

        x = x.view(-1, 64*2*8)

        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)

        return x

In [11]:
# Создаем объект для считывания потока с веб-камеры(обычно вебкамера идет под номером 0. иногда 1)
cap = cv2.VideoCapture(0) 

# Класс детектирования и обработки лица с веб-камеры 
class FaceDetector(object):

    def __init__(self, mtcnn, hand_model):
        self.mtcnn = mtcnn
        self.hand_model = hand_model
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    # Функция рисования найденных параметров на кадре
    def _draw_face(self, frame, boxes, probs, landmarks):
        try:
            for box, prob, ld in zip(boxes, probs, landmarks):
                # Рисуем обрамляющий прямоугольник лица на кадре
                cv2.rectangle(frame,
                              (box[0], box[1]),
                              (box[2], box[3]),
                              (0, 255, 0),
                              thickness=2)
        except:
            print('Something wrong in _draw_face function!')

        return frame
    
    # Функция текста предсказания на кадре
    def _draw_hand_pred(self, frame, hand_prediction):
        try:
            # пишем на кадре жест
            cv2.putText(frame, hand_prediction, (0, 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        except:
            print('Something wrong in _draw_hand_pred function!')

        return frame
    
    def _prepare_image(self, frame):
        img = Image.fromarray(frame).convert('L')
        img = img.resize((160, 60))
        img = torch.Tensor(np.array(img))
        img = img.unsqueeze(0).unsqueeze(0)
        return img
    
    # Функция в которой будет происходить процесс считывания и обработки каждого кадра
    def run(self):              
        # Заходим в бесконечный цикл
        while True:
            # Считываем каждый новый кадр - frame
            # ret - логическая переменая. Смысл - считали ли мы кадр с потока или нет
            ret, frame = cap.read()
            try:
                
                # детектируем расположение лица на кадре
                boxes, probs, landmarks = self.mtcnn.detect(frame, landmarks=True)
                if boxes is not None:
                    # Рисуем на кадре
                    self._draw_face(frame, boxes, probs, landmarks)                
                
                # распознаем жест
                img_tensor = self._prepare_image(frame)
                hand_output = self.hand_model(img_tensor)
                _, hand_pred_idx = torch.max(hand_output, 1)
                hand_prediction = hand_dict[hand_pred_idx[0].item()]
                self._draw_hand_pred(frame, hand_prediction)

            except:
                print('Something wrong im main cycle!')

            # Показываем кадр в окне, и назвываем его(окно) - 'Face Detection'
            cv2.imshow('Face Detection', frame)
            
            # Функция, которая проверяет нажатие на клавишу 'q'
            # Если нажатие произошло - выход из цикла. Конец работы приложения
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
        # Очищаем все объекты opencv, что мы создали
        cap.release()
        cv2.destroyAllWindows()
        
        
# Загружаем мтцнн
mtcnn = MTCNN()

# загружаем модель с жестами
hand_model = torch.load('hand_model.pth')
hand_model.eval()

# Создаем объект нашего класса приложения
fcd = FaceDetector(mtcnn, hand_model)

# Запускаем
fcd.run()